# Final Project - THE BATTLE OF NEIGHBOURHOOD
## Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem

This project aims to select or to suggest the best place in <b>India</b> for the targeted people to get settled. 

The report will be targeted to people who are looking to <b>settle in India.</b>

For settlement, or to find a neighbourhood to hunt for an apartment, we will be focusing on the <b>four metropolitan cities</b> only - namely, <b>Delhi, Mumbai, Chennai and Kolkata.</b>

We will explore the neighbourhood of all four cities and finally cluster the neighbourhoods using k-mean clustering.

We will suggest different cities on basis of their preferences and the atmosphere.

# Data

Based on the definition of the problem,factors that will influence our decisons is "Most common places in Neighbourhood of the cities"

Following data sources will be used to extract/generate the information:-

Part1:Preprocessing a data set to get pin codes and location information from https://data.gov.in/resources/all-india-pincode-directory-contact-details-along-latitude-and-longitude 

Part2: Getting coordinates of some places that are not included in dataset using Google Maps API Geocoding

Part3: Using Foursquare API to get neighbourhood data.

### Importing Required Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment', None)

import json # library to handle JSON files

import re#Library to find particular pattern
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import time #To create delays
# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Preprocessing the dataset

Loading Data in pandas dataframe form downloaded file

Dataset Location:-https://data.gov.in/resources/all-india-pincode-directory-contact-details-along-latitude-and-longitude 

This is the dataset prepared by Indian Government that contains pincode directory of all areas in India

### Reading the dataset

In [2]:
# Reading in the Data
main_df=pd.read_csv("Desktop/all_india_PO_list_without_APS_offices_ver2_lat_long.csv")

In [3]:
# View top 5 rows of the dataset
main_df.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
0,Achalapur B.O,504273,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Rechini S.O,Mancherial H.O,NaN,NaN
1,Ada B.O,504293,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Asifabad S.O,Mancherial H.O,NaN,NaN
2,Adegaon B.O,504307,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Boath,Adilabad,TELANGANA,NaN,Echoda S.O,Adilabad H.O,NaN,NaN
3,Adilabad Collectorate S.O,504001,S.O,Non-Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226703,NaN,Adilabad H.O,NaN,NaN
4,Adilabad H.O,504001,H.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226738,NaN,NaN,NaN,NaN


### Removing unnecessary Columns

In [4]:
# A list of required columns
required_columns=['pincode','Taluk','longitude','latitude']

In [5]:
# Making a new dataframe from old one extracting the required dataframes only
df_trun=main_df[required_columns]

In [6]:
# View top 5 rows of the dataframe
df_trun.head()

,pincode,Taluk,longitude,latitude
0,504273,Asifabad,NaN,NaN
1,504293,Asifabad,NaN,NaN
2,504307,Boath,NaN,NaN
3,504001,Adilabad,NaN,NaN
4,504001,Adilabad,NaN,NaN


In [7]:
# Checking No. of entries in the dataframe
df_trun.shape

(154797, 4)

### Removing unnecessary entries

Here the dataset contains approx 1.5 lakh rows but we want the data of metropolitan cities only.

In [8]:
#Making a new data frame containing all relevant entries
df_metro=df_trun[df_trun.Taluk.str.contains('Kolkata',na=False) | df_trun.Taluk.str.contains('Delhi',na=False) \
                 | df_trun.Taluk.str.contains('Chennai',na=False) | df_trun.Taluk.str.contains('Mumbai',na=False)]

In [9]:
# Viewing first five rows of dataset
df_metro.head()

,pincode,Taluk,longitude,latitude
32386,110090,East Delhi,NaN,NaN
32398,110053,East Delhi,NaN,NaN
32439,110094,Delhi North East,NaN,NaN
32459,110006,Delhi,NaN,NaN
32460,110033,Delhi,NaN,NaN


In [10]:
#Checking Shape of Dataset
df_metro.shape

(800, 4)

In [11]:
# Checking for missing values of Pincode
df_metro.isnull()['pincode'].value_counts()

False    800
Name: pincode, dtype: int64

In [12]:
# Verifying if the pincode is not repeated (They wll be required to fill the missing data)
len(df_metro['pincode'].unique())

234

In [13]:
#Dropping Duplicate Entries
df_metro.drop_duplicates(subset ="pincode",inplace=True)

In [14]:
#Resetting the index
df_metro.reset_index(drop=True,inplace=True)
#Viewing first 10 rows of dataframe
df_metro.head(10)

,pincode,Taluk,longitude,latitude
0,110090,East Delhi,NaN,NaN
1,110053,East Delhi,NaN,NaN
2,110094,Delhi North East,NaN,NaN
3,110006,Delhi,NaN,NaN
4,110033,Delhi,NaN,NaN
5,110036,Delhi,NaN,NaN
6,110034,Delhi,NaN,NaN
7,110052,Delhi,NaN,NaN
8,110039,Delhi,NaN,NaN
9,110042,Delhi,NaN,NaN


In [15]:
df_metro.shape

(234, 4)

### Filling missing values

In [16]:
# Checking for missing values of Coordinates
df_metro.isnull()['longitude'].value_counts()

True    234
Name: longitude, dtype: int64

Here we don't have Coordinates for any of the entry.So we have to fetch them using Google Geocoding API.

#### Using Geopy to get the location coordinates

In [17]:
geolocator = Nominatim(user_agent="kb636an@gmail.com")
def get_coords(pincode):
    location = geolocator.geocode(pincode)
    latitude = location.latitude
    longitude = location.longitude
    address = location.address
    name = re.findall('^.+Tehsil',location.address)
    if len(name)==0:
        name=address
    else:
        name=name[0]
    return name,latitude,longitude

#### Filling the dataframe with new values

In [18]:
#Function to get required data
for i in range(len(df_metro)):
    pincode=df_metro.loc[i]['pincode']
    try:
        data=get_coords(pincode)
        df_metro.loc[i,'Neighborhood']=data[0]
        df_metro.loc[i,'latitude']=data[1]
        df_metro.loc[i,'longitude']=data[2]
    except:
        pass

In [19]:
#Viewing top 5 rows of the dataframe
df_metro.head()

,pincode,Taluk,longitude,latitude,Neighborhood
0,110090,East Delhi,-34.926281,-8.016630,"110090, Rua Nova Descoberta, Bolo de Noiva, No..."
1,110053,East Delhi,77.279820,28.691913,"Babarpur, Shahdara Tehsil"
2,110094,Delhi North East,77.272664,28.716206,Yamuna Vihar Tehsil
3,110006,Delhi,77.231623,28.655984,"Chandni Chowk, Old Delhi, Delhi, Kotwali Tehsil"
4,110033,Delhi,77.168477,28.727506,"Jahangirpuri Colony, Model Town Tehsil"


In [20]:
#Saving the data to csv
df_metro.to_csv("India_Metropolitan.csv")

Here we are seaching for coordinates in India but some entries contain garbage values.

We will drop all the rows that contain inappropriate entries

#### Dropping entries with inappropriate entries

In [21]:
df_metro

,pincode,Taluk,longitude,latitude,Neighborhood
0,110090,East Delhi,-34.926281,-8.016630,"110090, Rua Nova Descoberta, Bolo de Noiva, No..."
1,110053,East Delhi,77.279820,28.691913,"Babarpur, Shahdara Tehsil"
2,110094,Delhi North East,77.272664,28.716206,Yamuna Vihar Tehsil
3,110006,Delhi,77.231623,28.655984,"Chandni Chowk, Old Delhi, Delhi, Kotwali Tehsil"
4,110033,Delhi,77.168477,28.727506,"Jahangirpuri Colony, Model Town Tehsil"
5,110036,Delhi,77.168828,28.817383,Alipur Tehsil
6,110034,Delhi,77.136121,28.694560,Saraswati Vihar Tehsil
7,110052,Delhi,77.175292,28.683588,"Ashok Vihar - IV, Saraswati Vihar Tehsil"
8,110039,Delhi,77.042110,28.797887,Narela Tehsil
9,110042,Delhi,77.107341,28.744966,"Sector 17, Rohini, Alipur Tehsil"


In [22]:
#The values that needs to be corrected are:-
drop_rows=[0,13,17,25,27,29,30,33,39,40,56,61,77,78,79,80,83,84,88,90,91,94,95,96,103,106,109,112,113,116,120,148,160,163,168,171,172,180,193,203,209,213,218,221]

In [23]:
#Dropping the garbage entries
df_metro.drop(df_metro.index[drop_rows],inplace=True)

In [24]:
#Resetting the index
df_metro.reset_index(drop=True,inplace=True)

In [25]:
# Viewing the first five rows of data frame
df_metro.head()

,pincode,Taluk,longitude,latitude,Neighborhood
0,110053,East Delhi,77.279820,28.691913,"Babarpur, Shahdara Tehsil"
1,110094,Delhi North East,77.272664,28.716206,Yamuna Vihar Tehsil
2,110006,Delhi,77.231623,28.655984,"Chandni Chowk, Old Delhi, Delhi, Kotwali Tehsil"
3,110033,Delhi,77.168477,28.727506,"Jahangirpuri Colony, Model Town Tehsil"
4,110036,Delhi,77.168828,28.817383,Alipur Tehsil


## Visualising the map

We will be using this function to create the map

In [26]:
def mapcreator(latitude,longitude,zoom,df):
    map_=folium.Map(location=[latitude,longitude],zoom_start=zoom)
    for lat, lng, location, neighborhood in zip(df['latitude'], df['longitude'], df['Taluk'], df['Neighborhood']):
        label = '{}, {}'.format(neighborhood, location)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_)
    return map_

In [27]:
#Finding Coordinates of India
_,latitude,longitude=get_coords('India')

In [28]:
map_india =mapcreator(latitude,longitude,5,df_metro)
map_india

## Modelling the Data

### Now,we will distribute the data in four different groups(on basis of the city they belong to) and perform the similar operations on all of them.

#### For Delhi

In [29]:
#Making a new dataframe containing areas of delhi only
df_delhi=df_metro[df_metro.Taluk.str.contains('Delhi',na=False)]

In [30]:
#Dropping pincode column as no longer needed
df_delhi.drop(['pincode'],axis=1,inplace=True)
#Viewing first 5 rows
df_delhi.head()

,Taluk,longitude,latitude,Neighborhood
0,East Delhi,77.279820,28.691913,"Babarpur, Shahdara Tehsil"
1,Delhi North East,77.272664,28.716206,Yamuna Vihar Tehsil
2,Delhi,77.231623,28.655984,"Chandni Chowk, Old Delhi, Delhi, Kotwali Tehsil"
3,Delhi,77.168477,28.727506,"Jahangirpuri Colony, Model Town Tehsil"
4,Delhi,77.168828,28.817383,Alipur Tehsil


In [31]:
#Checking No. of Entries
df_delhi.shape

(58, 4)

Let's get coordinates of delhi

In [32]:
#Finding Coordinates of Delhi
_,latitude,longitude=get_coords('Delhi,India')

Creating a map of Delhi

In [33]:
map_delhi =mapcreator(latitude,longitude,11,df_delhi)
map_delhi

#### For Mumbai

In [34]:
#Making a new dataframe containing areas of delhi only
df_mumbai=df_metro[df_metro.Taluk.str.contains('Mumbai',na=False)]

In [35]:
#Dropping pincode column as no longer needed
df_mumbai.drop(['pincode'],axis=1,inplace=True)
#Viewing first 5 rows
df_mumbai.head()

,Taluk,longitude,latitude,Neighborhood
58,Mumbai,72.867622,19.023074,"F/N Ward, Zone 2, Mumbai, Mumbai City, Maharas..."
59,Mumbai,72.842493,18.996311,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas..."
60,Mumbai,72.840388,18.981780,"E Ward, Zone 1, Mumbai, Mumbai City, Maharasht..."
61,Mumbai,72.846936,19.010619,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas..."
62,Mumbai,72.837844,18.968523,"E Ward, Zone 1, Mumbai, Mumbai City, Maharasht..."


In [36]:
#Checking No. of Entries
df_mumbai.shape

(35, 4)

Let's get coordinates of mumbai

In [37]:
#Finding Coordinates of Mumbai
_,latitude,longitude=get_coords('Mumbai,India')

Creating a map of Mumbai

In [38]:
map_mumbai =mapcreator(latitude,longitude,11,df_mumbai)
map_mumbai

#### For Chennai

In [39]:
#Making a new dataframe containing areas of delhi only
df_chennai=df_metro[df_metro.Taluk.str.contains('Chennai',na=False)]

In [40]:
#Dropping pincode column as no longer needed
df_chennai.drop(['pincode'],axis=1,inplace=True)
#Viewing first 5 rows
df_chennai.head()

,Taluk,longitude,latitude,Neighborhood
93,Chennai,80.267927,13.071184,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C..."
94,Chennai,80.262137,13.052888,"Ward 118, Zone 9 Teynampet, சென்னை - Chennai, ..."
95,Chennai,80.225850,13.064678,"Ward 109, Zone 9 Teynampet, சென்னை - Chennai, ..."
96,Chennai,80.256225,13.048387,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ..."
97,Chennai,80.258069,13.058448,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ..."


In [41]:
#Checking No. of Entries
df_chennai.shape

(32, 4)

Let's get coordinates of chennai

In [42]:
#Finding Coordinates of Delhi
_,latitude,longitude=get_coords('Chennai,India')

Creating a map of Chennai

In [43]:
map_chennai =mapcreator(latitude,longitude,11,df_chennai)
map_chennai

#### For Kolkata

In [44]:
#Making a new dataframe containing areas of delhi only
df_kolkata=df_metro[df_metro.Taluk.str.contains('Kolkata',na=False)]

In [45]:
#Dropping pincode column as no longer needed
df_kolkata.drop(['pincode'],axis=1,inplace=True)
#Viewing first 5 rows
df_kolkata.head()

,Taluk,longitude,latitude,Neighborhood
125,Kolkata,88.329496,22.530436,"Kolkata, West Bengal, 700027, India"
126,Kolkata,88.358539,22.583270,"Kolkata, West Bengal, 700007, India"
127,Kolkata,88.348456,22.537944,"Kolkata, West Bengal, 700020, India"
128,Kolkata,88.379909,22.472658,"Sonarpur, South 24 Parganas, West Bengal, 7000..."
129,Kolkata,88.329553,22.551956,"Kolkata, West Bengal, 700022, India"


In [46]:
#Checking No. of Entries
df_kolkata.shape

(65, 4)

Let's get coordinates of kolkata

In [47]:
#Finding Coordinates of Kolkata
_,latitude,longitude=get_coords('Kolkata,India')

Creating a map of Kolkata

In [48]:
map_kolkata =mapcreator(latitude,longitude,11,df_kolkata)
map_kolkata

<b>Defining FourSquare Credentials</b>

In [49]:
CLIENT_ID = 'XQIP25QCLDT0ZQ5N1UGMXKRWU3WYWLNQF34MBU4HCKVWFTQJ' # your Foursquare ID
CLIENT_SECRET = 'PIVB4SW3YZNVO50EUTCV4GJYSNIKX5GXRWNB24IR2BWQDQY0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XQIP25QCLDT0ZQ5N1UGMXKRWU3WYWLNQF34MBU4HCKVWFTQJ
CLIENT_SECRET:PIVB4SW3YZNVO50EUTCV4GJYSNIKX5GXRWNB24IR2BWQDQY0


<b> Let's explore the first Neighborhood in Delhi</b>

Get Neighborhood's Name

In [50]:
df_delhi.loc[0, 'Neighborhood']

'Babarpur, Shahdara Tehsil'

In [51]:
neighborhood_latitude = df_delhi.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df_delhi.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = df_delhi.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Babarpur, Shahdara Tehsil are 28.6919134, 77.2798204134402.


<b>No, let's get top 125 venues that are in Babarpur,Shahdra with radius of 500 meters</b>

In [52]:
# type your answer here
# type your answer here
radius=500
LIMIT=125
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Send the GET request and examine the results

In [53]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed2fede71c428001b93c0f9'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Delhi',
  'headerFullLocation': 'Delhi',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 28.696413404500007,
    'lng': 77.28494071797424},
   'sw': {'lat': 28.687413395499995, 'lng': 77.27470010890617}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5beca792b23dfa002b29932e',
       'name': 'Royal Enfield Service Center',
       'location': {'address': 'Plot No 201/6',
        'crossStreet': 'Main Road Number 66',
        'lat': 28.69078,
        'lng': 77.2786,
        'labeledLatLngs'

In [54]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [55]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Royal Enfield Service Center,Motorcycle Shop,28.690780,77.278600
1,yamuna vihar,Park,28.689816,77.283876


## Explore Neighborhoods

#### Let's create a function to repeat the same process to all the neighborhoods in that city

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            continue
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Venues in Delhi

In [57]:
delhi_venues = getNearbyVenues(names=df_delhi['Neighborhood'],
                                   latitudes=df_delhi['latitude'],
                                   longitudes=df_delhi['longitude']
                                  )

Babarpur, Shahdara Tehsil
Yamuna Vihar Tehsil
Chandni Chowk, Old Delhi, Delhi, Kotwali Tehsil
Jahangirpuri Colony, Model Town Tehsil
Alipur Tehsil
Saraswati Vihar Tehsil
Ashok Vihar - IV, Saraswati Vihar Tehsil
Narela Tehsil
Sector 17, Rohini, Alipur Tehsil
Banker, Narela Tehsil
Rohini Tehsil
Civil Lines Tehsil
Kanjhawalan Tehsil
Civil Lines, Lucknow, Sadar, Lucknow, Uttar Pradesh, 110054, India
Dhaka, Model Town Tehsil
Model Town Tehsil
North Delhi, Delhi, 110082, India
Mangolpuri Colony, Rohini Tehsil
Sector 7, Rohini, Alipur Tehsil
West Delhi, Delhi, 110056, India
Daryaganj, Old Delhi, Delhi, Kotwali Tehsil
Sadar Bazar, Delhi, Kotwali Tehsil
Chanakya Puri Tehsil
Chanakya Puri Tehsil
Rajendra Nagar, Karol Bagh Tehsil
Chanakya Puri Tehsil
Madangir, Ambedkar Nagar, Hauz Khas Tehsil
Kalkaji, Kalkaji Tehsil
Greater Kailash II, Kalkaji Tehsil
Defence Colony, Delhi, Defence Colony Teshil, South East Delhi, Delhi, 110024, India
Neeti Bagh, Hauz Khas Tehsil
South East Delhi, Delhi, 110013, I

Cheking Shape and the resulting dataframe

In [59]:
print(delhi_venues.shape)
delhi_venues.head()

(316, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Babarpur, Shahdara Tehsil",28.691913,77.279820,Royal Enfield Service Center,28.690780,77.278600,Motorcycle Shop
1,"Babarpur, Shahdara Tehsil",28.691913,77.279820,yamuna vihar,28.689816,77.283876,Park
2,Yamuna Vihar Tehsil,28.716206,77.272664,Axis Bank ATM,28.715680,77.275780,ATM
3,Yamuna Vihar Tehsil,28.716206,77.272664,The Bolt Agency,28.717938,77.270129,Business Service
4,Yamuna Vihar Tehsil,28.716206,77.272664,Axis Bank ATM,28.720010,77.274280,ATM


Checking count of venues for each neighborhood

In [60]:
delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Ashok Vihar - IV, Saraswati Vihar Tehsil",2,2,2,2,2,2
"Babarpur, Shahdara Tehsil",2,2,2,2,2,2
"Banker, Narela Tehsil",1,1,1,1,1,1
Chanakya Puri Tehsil,100,100,100,100,100,100
"Chandni Chowk, Old Delhi, Delhi, Kotwali Tehsil",13,13,13,13,13,13
"Chhawala, Kapashera Tehsil",1,1,1,1,1,1
Civil Lines Tehsil,2,2,2,2,2,2
"Civil Lines, Lucknow, Sadar, Lucknow, Uttar Pradesh, 110054, India",2,2,2,2,2,2
"Daryaganj, Old Delhi, Delhi, Kotwali Tehsil",4,4,4,4,4,4


##### Let's find out how many unique categories can be curated from all the returned venues

In [61]:
print('There are {} uniques categories.'.format(len(delhi_venues['Venue Category'].unique())))

There are 95 uniques categories.


#### Venues in Mumbai

In [62]:
mumbai_venues = getNearbyVenues(names=df_mumbai['Neighborhood'],
                                   latitudes=df_mumbai['latitude'],
                                   longitudes=df_mumbai['longitude']
                                  )

F/N Ward, Zone 2, Mumbai, Mumbai City, Maharashtra, 400037, India
F/S Ward, Zone 2, Mumbai, Mumbai City, Maharashtra, 400012, India
E Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400033, India
F/S Ward, Zone 2, Mumbai, Mumbai City, Maharashtra, 400014, India
E Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400010, India
F/S Ward, Zone 2, Mumbai, Mumbai City, Maharashtra, 400031, India
B Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400009, India
K/E Ward, Zone 3, Mumbai, Mumbai Suburban, Maharashtra, 400069, India
K/W Ward, Zone 3, Mumbai, Mumbai Suburban, Maharashtra, 400053, India
Bandra West, Zone 3, Mumbai, Mumbai Suburban, Maharashtra, 400050, India
K/E Ward, Zone 3, Mumbai, Mumbai Suburban, Maharashtra, 400093, India
H/W Ward, Zone 3, Mumbai, Mumbai Suburban, Maharashtra, 400052, India
Panvel, Raigad, Maharashtra, 400056, India
K/W Ward, Zone 3, Mumbai, Mumbai Suburban, Maharashtra, 400061, India
H/W Ward, Zone 3, Mumbai, Mumbai Suburban, Maharashtra, 400054, India
F/N Ward

Cheking Shape and the resulting dataframe

In [64]:
print(mumbai_venues.shape)
mumbai_venues.head()

(576, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"F/N Ward, Zone 2, Mumbai, Mumbai City, Maharas...",19.023074,72.867622,Club House,19.023104,72.865814,Gym / Fitness Center
1,"F/N Ward, Zone 2, Mumbai, Mumbai City, Maharas...",19.023074,72.867622,VIT Grounds,19.021006,72.870399,Playground
2,"F/N Ward, Zone 2, Mumbai, Mumbai City, Maharas...",19.023074,72.867622,Palkhi Restaurant,19.022163,72.863034,Indian Restaurant
3,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",18.996311,72.842493,ITC Grand Central,18.998469,72.838433,Hotel
4,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",18.996311,72.842493,Terrace Garden,18.998119,72.838529,Roof Deck


Checking count of venues for each neighborhood

In [65]:
mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"A Ward, Zone 1, Mumbai, Mumbai City, 400032, India",55,55,55,55,55,55
"A Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400001, India",12,12,12,12,12,12
"A Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400005, India",14,14,14,14,14,14
"B Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400009, India",8,8,8,8,8,8
"Bandra West, Zone 3, Mumbai, Mumbai Suburban, Maharashtra, 400050, India",47,47,47,47,47,47
"C Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400002, India",8,8,8,8,8,8
"D Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400007, India",16,16,16,16,16,16
"D Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400026, India",21,21,21,21,21,21
"D Ward, Zone 1, Mumbai, Mumbai City, Maharashtra, 400034, India",16,16,16,16,16,16


##### Let's find out how many unique categories can be curated from all the returned venues

In [66]:
print('There are {} uniques categories.'.format(len(mumbai_venues['Venue Category'].unique())))

There are 133 uniques categories.


#### Venues in Chennai

In [67]:
chennai_venues = getNearbyVenues(names=df_chennai['Neighborhood'],
                                   latitudes=df_chennai['latitude'],
                                   longitudes=df_chennai['longitude']
                                  )

Ward 63, Zone 5 Royapuram, சென்னை - Chennai, Chennai district, Tamil Nadu, 600002, India
Ward 118, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600005, India
Ward 109, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600094, India
Ward 111, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600086, India
Ward 111, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600006, India
Ward 136, Zone 10 Kodambakkam, சென்னை - Chennai, Chennai district, Tamil Nadu, 600017, India
Ward 120, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600014, India
Ward 113, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600034, India
Ward 123, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600004, India
Ward 123, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600018, India
Ward 127, Zone 10 Kodambakkam, சென்னை - Chennai, Chennai district, Tamil Nadu, 600107, India
Ward 

Cheking Shape and the resulting dataframe

In [68]:
print(chennai_venues.shape)
chennai_venues.head()

(305, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",13.071184,80.267927,Ritche Street,13.069477,80.271374,Flea Market
1,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",13.071184,80.267927,Hotel Saravana Bhavan,13.068072,80.271226,Indian Restaurant
2,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",13.071184,80.267927,Delta Peripherals,13.069419,80.268560,Electronics Store
3,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",13.071184,80.267927,Pudupet Market,13.067733,80.267329,Market
4,"Ward 118, Zone 9 Teynampet, சென்னை - Chennai, ...",13.052888,80.262137,Hotel Ponnusamy,13.049504,80.262197,Indian Restaurant


Checking count of venues for each neighborhood

In [69]:
chennai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Ward 109, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600094, India",4,4,4,4,4,4
"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600006, India",30,30,30,30,30,30
"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600086, India",30,30,30,30,30,30
"Ward 113, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600034, India",14,14,14,14,14,14
"Ward 118, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600005, India",2,2,2,2,2,2
"Ward 120, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600014, India",4,4,4,4,4,4
"Ward 122, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600035, India",4,4,4,4,4,4
"Ward 123, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600004, India",13,13,13,13,13,13
"Ward 123, Zone 9 Teynampet, சென்னை - Chennai, Chennai district, Tamil Nadu, 600018, India",16,16,16,16,16,16


##### Let's find out how many unique categories can be curated from all the returned venues

In [70]:
print('There are {} uniques categories.'.format(len(chennai_venues['Venue Category'].unique())))

There are 89 uniques categories.


#### Venues in Kolkata

In [71]:
kolkata_venues = getNearbyVenues(names=df_kolkata['Neighborhood'],
                                   latitudes=df_kolkata['latitude'],
                                   longitudes=df_kolkata['longitude']
                                  )

Kolkata, West Bengal, 700027, India
Kolkata, West Bengal, 700007, India
Kolkata, West Bengal, 700020, India
Sonarpur, South 24 Parganas, West Bengal, 700086, India
Kolkata, West Bengal, 700022, India
Kolkata, West Bengal, 700008, India
Kolkata, West Bengal, 700018, India
Kolkata, West Bengal, 700034, India
Kolkata, West Bengal, 700025, India
Kolkata, West Bengal, 700032, India
Kolkata, West Bengal, 700042, India
Dhakuria, Kolkata, West Bengal, 700029, India
Kasbah North, Kolkata, West Bengal, 700107, India
Kolkata, West Bengal, 700047, India
Kolkata, West Bengal, 700024, India
Sonarpur, South 24 Parganas, West Bengal, 700075, India
Kolkata, West Bengal, 700095, India
Kolkata, West Bengal, 700078, India
Kolkata, West Bengal, 700033, India
Behala, Kolkata, West Bengal, 700061, India
Kolkata, West Bengal, 700068, India
Kolkata, West Bengal, 700053, India
Thakurpukur Mahestola, South 24 Parganas, West Bengal, 700063, India
Kolkata, West Bengal, 700026, India
Kolkata, West Bengal, 700023, I

Cheking Shape and the resulting dataframe

In [72]:
print(kolkata_venues.shape)
kolkata_venues.head()

(441, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Kolkata, West Bengal, 700027, India",22.530436,88.329496,Garden Cafe,22.532051,88.331752,South Indian Restaurant
1,"Kolkata, West Bengal, 700027, India",22.530436,88.329496,Drive Inn Snack Bar,22.529316,88.331704,Vegetarian / Vegan Restaurant
2,"Kolkata, West Bengal, 700027, India",22.530436,88.329496,The Agri-Horticultural Society of India,22.527185,88.332787,Garden
3,"Kolkata, West Bengal, 700007, India",22.583270,88.358539,Jorasankho Thakur-Bari,22.584401,88.359526,History Museum
4,"Kolkata, West Bengal, 700007, India",22.583270,88.358539,Marble Palace,22.582444,88.360185,Art Museum


Checking count of venues for each neighborhood

In [74]:
kolkata_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Baguiati, Rajarhat, North 24 Parganas, West Bengal, 700052, India",6,6,6,6,6,6
"Barrackpur - II, North 24 Parganas, West Bengal, 700028, India",5,5,5,5,5,5
"Barrackpur - II, North 24 Parganas, West Bengal, 700035, India",1,1,1,1,1,1
"Barrackpur - II, North 24 Parganas, West Bengal, 700065, India",2,2,2,2,2,2
"Barrackpur - II, North 24 Parganas, West Bengal, 700074, India",5,5,5,5,5,5
"Barrackpur - II, North 24 Parganas, West Bengal, 700080, India",4,4,4,4,4,4
"Barrackpur - II, North 24 Parganas, West Bengal, 700090, India",3,3,3,3,3,3
"Behala, Kolkata, West Bengal, 700060, India",3,3,3,3,3,3
"Behala, Kolkata, West Bengal, 700061, India",2,2,2,2,2,2


##### Let's find out how many unique categories can be curated from all the returned venues

In [75]:
print('There are {} uniques categories.'.format(len(kolkata_venues['Venue Category'].unique())))

There are 98 uniques categories.


### Analysing Neighborhoods in Delhi

In [77]:
# one hot encoding
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhood'] = delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_onehot.head()

,Neighborhood,ATM,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Garden,Bistro,Bookstore,Boutique,Brewery,Burger Joint,Bus Station,Business Service,Café,Candy Store,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gift Shop,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,North Indian Restaurant,Northeast Indian Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Portuguese Restaurant,Pub,Restaurant,Road,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,"Babarpur, Shahdara Tehsil",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Babarpur, Shahdara Tehsil",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Yamuna Vihar Tehsil,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Yamuna Vihar Tehsil,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Yamuna Vihar Tehsil,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [78]:
# Examining Shape of new dataframe
delhi_onehot.shape

(316, 96)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [79]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped

,Neighborhood,ATM,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Garden,Bistro,Bookstore,Boutique,Brewery,Burger Joint,Bus Station,Business Service,Café,Candy Store,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gift Shop,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,North Indian Restaurant,Northeast Indian Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Portuguese Restaurant,Pub,Restaurant,Road,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,"Ashok Vihar - IV, Saraswati Vihar Tehsil",0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.500000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.500000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000
1,"Babarpur, Shahdara Tehsil",0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.5,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.500000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000
2,"Banker, Narela Tehsil",0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,1.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000
3,Chanakya Puri Tehsil,0.00,0.000000,0.01,0.040000,0.0,0.020000,0.000000,0.060000,0.01,0.03,0.02,0.0,0.01,0.000000,0.0,0.00,0.130000,0.0,0.030000,0.010000,0.080000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.010000,0.01,0.0,0.01,0.040000,0.010000,0.010000,0.000000,0.000000,0.010000,0.01,0.000000,0.010000,0.010000,0.01,0.01,0.01,0.00,0.020000,0.00,0.000000,0.070000,0.01,0.01,0.030000,0.000000,0.000000,0.030000,0.010000,0.03,0.01,0.000000,0.01,0.000000,0.000000,0.0,0.0,0.01,0.000000,0.000000,0.0,0.02,

In [80]:
#Function to sort venues in descending order
#We will use this function in futyre too
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_delhi = pd.DataFrame(columns=columns)
neighborhoods_venues_delhi['Neighborhood'] = delhi_grouped['Neighborhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_delhi.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_delhi.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Ashok Vihar - IV, Saraswati Vihar Tehsil",Market,Indian Restaurant,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
1,"Babarpur, Shahdara Tehsil",Park,Motorcycle Shop,Women's Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop
2,"Banker, Narela Tehsil",Electronics Store,Women's Store,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
3,Chanakya Puri Tehsil,Café,Coffee Shop,Indian Restaurant,Bar,Pub,Fast Food Restaurant,Asian Restaurant,Italian Restaurant,Lounge,Mediterranean Restaurant
4,"Chandni Chowk, Old Delhi, Delhi, Kotwali Tehsil",Indian Restaurant,Market,Snack Place,Fast Food Restaurant,Hotel,Dessert Shop,Paper / Office Supplies Store,Flea Market,Deli / Bodega,Department Store


### Analysing Neighborhoods in Mumbai

In [82]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lounge,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Outlet Mall,Park,Pharmacy,Pizza Place,Platform,Playground,Pool,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Spanish Restaurant,Sports Bar,Stadium,Steakhouse,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,"F/N Ward, Zone 2, Mumbai, Mumbai City, Maharas...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"F/N Ward, Zone 2, Mumbai, Mumbai City, Maharas...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"F/N Ward, Zone 2, Mumbai, Mumbai City, Maharas...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [83]:
# Examining Shape of new dataframe
mumbai_onehot.shape

(576, 134)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [84]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped

,Neighborhood,Accessories Store,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lounge,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Outlet Mall,Park,Pharmacy,Pizza Place,Platform,Playground,Pool,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Spanish Restaurant,Sports Bar,Stadium,Steakhouse,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,"A Ward, Zone 1, Mumbai, Mumbai City, 400032, I...",0.000000,0.000000,0.000000,0.036364,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.0000,0.00,0.000000,0.018182,0.018182,0.000000,0.018182,0.000000,0.000000,0.000000,0.090909,0.000000,0.036364,0.018182,0.000000,0.036364,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000,0.018182,0.000000,0.054545,0.018182,0.000000,0.018182,0.0000,0.072727,0.018182,0.000000,0.000,0.00000,0.000000,0.0000,0.000000,0.018182,0.000000,0.000000,0.00000,0.0,0.018182,0.000000,0.018182,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.018182,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.163636,0.000,0.018182,0.000000,0.018182,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.018182,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.018182,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.018182,0.018182,0.018182,0.000000,0.018182,0.000000,0.000000,0.018182,0.018182,0.000000
1,"A Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.083333,0.000000,0.0,0.000000,0.000000,0.00000,0.083333,0.000000,0.083333,0.000000,0.083333,0.000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [136]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_mumbai = pd.DataFrame(columns=columns)
neighborhoods_venues_mumbai['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_mumbai.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_mumbai.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"A Ward, Zone 1, Mumbai, Mumbai City, 400032, I...",Indian Restaurant,Café,Fast Food Restaurant,Dessert Shop,Chinese Restaurant,Art Gallery,Coffee Shop,Nightclub,Japanese Restaurant,Breakfast Spot
1,"A Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",Italian Restaurant,Pizza Place,Indian Restaurant,Ice Cream Shop,Bar,Snack Place,Chinese Restaurant,Spa,Hotel,Thai Restaurant
2,"A Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",Spa,Chinese Restaurant,German Restaurant,Italian Restaurant,Toy / Game Store,Gym,Thai Restaurant,Coffee Shop,Men's Store,Indian Restaurant
3,"B Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",Indian Restaurant,Furniture / Home Store,Chinese Restaurant,Indian Sweet Shop,Flea Market,Deli / Bodega,Department Store,Dessert Shop,Cupcake Shop,Fish Market
4,"Bandra West, Zone 3, Mumbai, Mumbai Suburban, ...",Indian Restaurant,Café,Coffee Shop,Chinese Restaurant,Gourmet Shop,Asian Restaurant,Snack Place,Bookstore,Clothing Store,Gym / Fitness Center


### Analysing Neighborhoods in Chennai

In [86]:
# one hot encoding
chennai_onehot = pd.get_dummies(chennai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chennai_onehot['Neighborhood'] = chennai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chennai_onehot.columns[-1]] + list(chennai_onehot.columns[:-1])
chennai_onehot = chennai_onehot[fixed_columns]

chennai_onehot.head()

,Neighborhood,ATM,African Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Bistro,Bookstore,Boutique,Bridal Shop,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Department Store,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food Court,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gift Shop,Golf Course,Grocery Store,Gym / Fitness Center,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,North Indian Restaurant,Office,Park,Pharmacy,Pizza Place,Platform,Pub,Record Shop,Restaurant,Rock Club,Russian Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,South Indian Restaurant,Spa,Sports Bar,Tea Room,Tennis Stadium,Theater,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Women's Store
0,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Ward 118, Zone 9 Teynampet, சென்னை - Chennai, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
# Examining Shape of new dataframe
chennai_onehot.shape

(305, 90)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [89]:
chennai_grouped = chennai_onehot.groupby('Neighborhood').mean().reset_index()
chennai_grouped

,Neighborhood,ATM,African Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Bistro,Bookstore,Boutique,Bridal Shop,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Department Store,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food Court,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gift Shop,Golf Course,Grocery Store,Gym / Fitness Center,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,North Indian Restaurant,Office,Park,Pharmacy,Pizza Place,Platform,Pub,Record Shop,Restaurant,Rock Club,Russian Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,South Indian Restaurant,Spa,Sports Bar,Tea Room,Tennis Stadium,Theater,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Women's Store
0,"Ward 109, Zone 9 Teynampet, சென்னை - Chennai, ...",0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ...",0.033333,0.000000,0.000000,0.000000,0.066667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.100000,0.033333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.100000,0.000000,0.000000,0.066667,0.033333,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.0,0.066667,0.0,0.166667,0.000000,0.000000,0.000000,0.000000,0.0000,0.033333,0.000000,0.033333,0.0,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000
2,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ...",0.000000,0.033333,0.033333,0.000000,0.000000,0.033333,0.0,0.000000,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.0,0.033333,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.033333,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.0,0.00,0.033333,0.233333,0.033333,0.033333,0.033333,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.033333,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333
3,"Ward 113, Zone 9 Teynampet, சென்னை - Chennai, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.071429,0.000000,0.000000,0.000000,0.00,0.000000,0.071429,0.000000,0.071429,0.000000,0.071429,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,

Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_chennai = pd.DataFrame(columns=columns)
neighborhoods_venues_chennai['Neighborhood'] = chennai_grouped['Neighborhood']

for ind in np.arange(chennai_grouped.shape[0]):
    neighborhoods_venues_chennai.iloc[ind, 1:] = return_most_common_venues(chennai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_chennai.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Ward 109, Zone 9 Teynampet, சென்னை - Chennai, ...",Burger Joint,Juice Bar,Bakery,Department Store,Hotel,Hotel Bar,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant
1,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ...",Multiplex,Indian Restaurant,Café,Bakery,Juice Bar,Middle Eastern Restaurant,Movie Theater,Restaurant,Bus Station,Chinese Restaurant
2,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ...",Indian Restaurant,Hotel,Department Store,Juice Bar,Jewelry Store,Italian Restaurant,Ice Cream Shop,Fast Food Restaurant,Concert Hall,Park
3,"Ward 113, Zone 9 Teynampet, சென்னை - Chennai, ...",Indian Restaurant,Bookstore,Italian Restaurant,Movie Theater,Clothing Store,Café,Bus Station,South Indian Restaurant,Park,Hotel
4,"Ward 118, Zone 9 Teynampet, சென்னை - Chennai, ...",Indian Restaurant,Record Shop,Food Court,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market


### Analysing Neighborhoods in Kolkata

In [91]:
# one hot encoding
kolkata_onehot = pd.get_dummies(kolkata_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kolkata_onehot['Neighborhood'] = kolkata_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kolkata_onehot.columns[-1]] + list(kolkata_onehot.columns[:-1])
kolkata_onehot = kolkata_onehot[fixed_columns]

kolkata_onehot.head()

,Vegetarian / Vegan Restaurant,ATM,American Restaurant,Art Museum,Asian Restaurant,Auto Dealership,Awadhi Restaurant,BBQ Joint,Bakery,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Bus Station,Cafeteria,Café,Chinese Restaurant,Circus,Clothing Store,Coffee Shop,Concert Hall,Cosmetics Shop,Cricket Ground,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food,Food Court,Food Stand,Furniture / Home Store,Garden,Gastropub,Grocery Store,Gym,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hotel,IT Services,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lounge,Market,Metro Station,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Mughlai Restaurant,Multiplex,Museum,Music Store,Neighborhood,Nightclub,Optical Shop,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Restaurant,River,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Train Station,Tram Station
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Kolkata, West Bengal, 700027, India",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Kolkata, West Bengal, 700027, India",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Kolkata, West Bengal, 700027, India",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Kolkata, West Bengal, 700007, India",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Kolkata, West Bengal, 700007, India",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [92]:
# Examining Shape of new dataframe
kolkata_onehot.shape

(441, 98)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [93]:
kolkata_grouped = kolkata_onehot.groupby('Neighborhood').mean().reset_index()
kolkata_grouped

,Neighborhood,Vegetarian / Vegan Restaurant,ATM,American Restaurant,Art Museum,Asian Restaurant,Auto Dealership,Awadhi Restaurant,BBQ Joint,Bakery,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Bus Station,Cafeteria,Café,Chinese Restaurant,Circus,Clothing Store,Coffee Shop,Concert Hall,Cosmetics Shop,Cricket Ground,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field,Flea Market,Food,Food Court,Food Stand,Furniture / Home Store,Garden,Gastropub,Grocery Store,Gym,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hotel,IT Services,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lounge,Market,Metro Station,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Mughlai Restaurant,Multiplex,Museum,Music Store,Nightclub,Optical Shop,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Restaurant,River,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Train Station,Tram Station
0,"Baguiati, Rajarhat, North 24 Parganas, West Be...",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.166667,0.166667,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Barrackpur - II, North 24 Parganas, West Benga...",0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Barrackpur - II, North 24 Parganas, West Benga...",0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Barrackpur - II, North 24 Parganas, West Benga...",0.000000,0.500000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000

Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_kolkata = pd.DataFrame(columns=columns)
neighborhoods_venues_kolkata['Neighborhood'] = kolkata_grouped['Neighborhood']

for ind in np.arange(kolkata_grouped.shape[0]):
    neighborhoods_venues_kolkata.iloc[ind, 1:] = return_most_common_venues(kolkata_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_kolkata.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Baguiati, Rajarhat, North 24 Parganas, West Be...",Pizza Place,Sandwich Place,Chinese Restaurant,Café,Electronics Store,Snack Place,Gym,Harbor / Marina,Cricket Ground,Department Store
1,"Barrackpur - II, North 24 Parganas, West Benga...",Restaurant,ATM,Performing Arts Venue,Indie Movie Theater,Tram Station,Food Court,Department Store,Dhaba,Dumpling Restaurant,Electronics Store
2,"Barrackpur - II, North 24 Parganas, West Benga...",ATM,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field,Flea Market
3,"Barrackpur - II, North 24 Parganas, West Benga...",ATM,Pharmacy,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
4,"Barrackpur - II, North 24 Parganas, West Benga...",ATM,IT Services,Café,Mughlai Restaurant,Food Stand,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field


## Cluster Neighborhoods

### In Delhi

In [95]:
# Running k-means to cluster
# set number of clusters
kclusters = 5

delhi_grouped_clustering = delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 4, 3, 1, 2, 4, 3, 3, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_delhi.insert(0, 'Cluster Labels', kmeans.labels_)

delhi_merged = df_delhi

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
delhi_merged = df_delhi.join(neighborhoods_venues_delhi.set_index('Neighborhood'), on='Neighborhood')
#drop values for which cluster labels is not defined
delhi_merged=delhi_merged.dropna()
#Converting cluster labels in integer
delhi_merged['Cluster Labels'] = delhi_merged['Cluster Labels'].apply(np.int64)
delhi_merged.head() # check the last columns!

Visualizing the cluster results

In [130]:
#Finding Coordinates of Delhi
_,latitude,longitude=get_coords('Delhi')

In [131]:
# create map
delhi_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(delhi_merged['latitude'], delhi_merged['longitude'], delhi_merged['Neighborhood'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(delhi_clusters)
       
delhi_clusters

### In Mumbai

In [132]:
# Running k-means to cluster
# set number of clusters
kclusters = 5

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 2, 0, 2, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [137]:
# add clustering labels
neighborhoods_venues_mumbai.insert(0, 'Cluster Labels', kmeans.labels_)

mumbai_merged = df_mumbai

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mumbai_merged = df_mumbai.join(neighborhoods_venues_mumbai.set_index('Neighborhood'), on='Neighborhood')

#drop values for which cluster labels is not defined
mumbai_merged=mumbai_merged.dropna()
#Converting cluster labels in integer
mumbai_merged['Cluster Labels'] = mumbai_merged['Cluster Labels'].apply(np.int64)


mumbai_merged.head() # check the last columns!

,Taluk,longitude,latitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Mumbai,72.867622,19.023074,"F/N Ward, Zone 2, Mumbai, Mumbai City, Maharas...",2,Gym / Fitness Center,Playground,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop,Diner,Department Store
59,Mumbai,72.842493,18.996311,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",0,Gym / Fitness Center,Hotel,Restaurant,Lounge,Indian Restaurant,Bank,Chinese Restaurant,Coffee Shop,Roof Deck,Dessert Shop
60,Mumbai,72.840388,18.981780,"E Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",4,Furniture / Home Store,Pizza Place,Hookah Bar,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop,Diner,Dessert Shop
61,Mumbai,72.846936,19.010619,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",0,Movie Theater,Cricket Ground,Indian Restaurant,Grocery Store,Historic Site,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Fish Market
62,Mumbai,72.837844,18.968523,"E Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",0,Indian Restaurant,Café,Garden,Ice Cream Shop,Chinese Restaurant,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop


Visualizing the cluster results

In [138]:
#Finding Coordinates of Mumbai
_,latitude,longitude=get_coords('Mumbai')

In [140]:
# create map
mumbai_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['latitude'], mumbai_merged['longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mumbai_clusters)
       
mumbai_clusters

### In Chennai

In [141]:
# Running k-means to cluster
# set number of clusters
kclusters = 5

chennai_grouped_clustering = chennai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chennai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 0, 0, 0, 0, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [142]:
# add clustering labels
neighborhoods_venues_chennai.insert(0, 'Cluster Labels', kmeans.labels_)

chennai_merged = df_chennai

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
chennai_merged = df_chennai.join(neighborhoods_venues_chennai.set_index('Neighborhood'), on='Neighborhood')

#drop values for which cluster labels is not defined
chennai_merged=chennai_merged.dropna()
#Converting cluster labels in integer
chennai_merged['Cluster Labels'] = chennai_merged['Cluster Labels'].apply(np.int64)

chennai_merged.head() # check the last columns!

,Taluk,longitude,latitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,Chennai,80.267927,13.071184,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",0,Indian Restaurant,Electronics Store,Flea Market,Market,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop
94,Chennai,80.262137,13.052888,"Ward 118, Zone 9 Teynampet, சென்னை - Chennai, ...",0,Indian Restaurant,Record Shop,Food Court,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market
95,Chennai,80.225850,13.064678,"Ward 109, Zone 9 Teynampet, சென்னை - Chennai, ...",1,Burger Joint,Juice Bar,Bakery,Department Store,Hotel,Hotel Bar,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant
96,Chennai,80.256225,13.048387,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ...",0,Indian Restaurant,Hotel,Department Store,Juice Bar,Jewelry Store,Italian Restaurant,Ice Cream Shop,Fast Food Restaurant,Concert Hall,Park
97,Chennai,80.258069,13.058448,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ...",1,Multiplex,Indian Restaurant,Café,Bakery,Juice Bar,Middle Eastern Restaurant,Movie Theater,Restaurant,Bus Station,Chinese Restaurant


Visualizing the cluster results

In [143]:
#Finding Coordinates of Chennai
_,latitude,longitude=get_coords('Chennai')

In [144]:
# create map
chennai_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chennai_merged['latitude'], chennai_merged['longitude'], chennai_merged['Neighborhood'], chennai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(chennai_clusters)
       
chennai_clusters

### In Kolkata

In [146]:
# Running k-means to cluster
# set number of clusters
kclusters = 5

kolkata_grouped_clustering = kolkata_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kolkata_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 1, 1, 1, 4, 3, 0, 3, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [147]:
# add clustering labels
neighborhoods_venues_kolkata.insert(0, 'Cluster Labels', kmeans.labels_)

kolkata_merged = df_kolkata

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kolkata_merged = df_kolkata.join(neighborhoods_venues_kolkata.set_index('Neighborhood'), on='Neighborhood')

#drop values for which cluster labels is not defined
kolkata_merged=kolkata_merged.dropna()
#Converting cluster labels in integer
kolkata_merged['Cluster Labels'] = kolkata_merged['Cluster Labels'].apply(np.int64)

kolkata_merged.head() # check the last columns!

,Taluk,longitude,latitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
125,Kolkata,88.329496,22.530436,"Kolkata, West Bengal, 700027, India",4,Vegetarian / Vegan Restaurant,Garden,South Indian Restaurant,Food Court,Cricket Ground,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant
126,Kolkata,88.358539,22.583270,"Kolkata, West Bengal, 700007, India",4,Indian Restaurant,History Museum,Art Museum,Museum,Food Court,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant
127,Kolkata,88.348456,22.537944,"Kolkata, West Bengal, 700020, India",4,Nightclub,Bengali Restaurant,American Restaurant,Café,Fast Food Restaurant,Department Store,Dhaba,Multiplex,Plaza,Restaurant
128,Kolkata,88.379909,22.472658,"Sonarpur, South 24 Parganas, West Bengal, 7000...",0,Tea Room,Circus,Electronics Store,Bus Station,Tram Station,Food Court,Department Store,Dhaba,Dumpling Restaurant,Fast Food Restaurant
129,Kolkata,88.329553,22.551956,"Kolkata, West Bengal, 700022, India",4,ATM,Historic Site,Food,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant


Visualizing the cluster results

In [148]:
#Finding Coordinates of Kolkata
_,latitude,longitude=get_coords('Kolkata')

In [149]:
# create map
kolkata_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kolkata_merged['latitude'], kolkata_merged['longitude'], kolkata_merged['Neighborhood'], kolkata_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(kolkata_clusters)
       
kolkata_clusters

## Analysis

### In Delhi

Analyse each of the clusters to identify the characteristics of each cluster and the neighborhoods in them.

<b>Examine the first cluster</b>

In [173]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 0, delhi_merged.columns[[3] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,"West Delhi, Delhi, 110056, India",Train Station,Women's Store,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant


First Cluster has only one neighbour. It contains Train Station,Restaurants and Stores.This Neighborhood might be intresting for those who travels with train often.

<b>Examine the second cluster</b>

In [172]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 1, delhi_merged.columns[[3] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Chandni Chowk, Old Delhi, Delhi, Kotwali Tehsil",Indian Restaurant,Market,Snack Place,Fast Food Restaurant,Hotel,Dessert Shop,Paper / Office Supplies Store,Flea Market,Deli / Bodega,Department Store
6,"Ashok Vihar - IV, Saraswati Vihar Tehsil",Market,Indian Restaurant,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
27,"Kalkaji, Kalkaji Tehsil",Fast Food Restaurant,Coffee Shop,Indian Restaurant,Pizza Place,Food,Electronics Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
32,"Sriniwaspuri, Defence Colony Teshil, South Eas...",Wine Bar,Hotel,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
34,"Defence Colony Teshil, South East Delhi, Delhi...",ATM,Fast Food Restaurant,Asian Restaurant,Indian Restaurant,Shopping Mall,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
40,"Sarojini Nagar, Vasant Vihar Tehsil",Indian Restaurant,Fast Food Restaurant,Women's Store,Department Store,Dessert Shop,Pizza Place,Shopping Mall,Market,Restaurant,Electronics Store
41,"Sector 5, Dwarka, Patel Nagar Tehsil",Fast Food Restaurant,Indian Restaurant,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
49,Dwarka Tehsil,Indian Restaurant,Bar,Women's Store,Flea Market,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
54,"Paschim Vihar, Punjabi Bagh Tehsil",Hotel,Coffee Shop,Indian Restaurant,Food Truck,Fast Food Restaurant,Women's Store,Event Space,Department Store,Dessert Shop,Dim Sum Restaurant
56,"Sector 13, Dwarka, Dwarka Tehsil",Fast Food Restaurant,Fried Chicken Joint,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant


It contains Market,Mall,ATM,Restaurants,Pizza Place and various other venues. This neighbourhood can be good for medium income families.These neighborhood may attract people who are foodies.

<b>Examine the third cluster</b>

In [171]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 2, delhi_merged.columns[[3] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Yamuna Vihar Tehsil,ATM,Business Service,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
7,Narela Tehsil,ATM,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store
17,"Mangolpuri Colony, Rohini Tehsil",ATM,Chinese Restaurant,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
44,"Chhawala, Kapashera Tehsil",ATM,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store
47,"Mansaram Park, Patel Nagar Tehsil",ATM,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store


Third cluster contains ATM,Restaurants Department Stores. These people are woking professionals and might intrest people who are relocating due to transfer.

<b>Examine the fourth cluster</b>

In [174]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 3, delhi_merged.columns[[3] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Babarpur, Shahdara Tehsil",Park,Motorcycle Shop,Women's Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop
5,Saraswati Vihar Tehsil,Hotel,IT Services,Spa,Diner,Women's Store,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
13,"Civil Lines, Lucknow, Sadar, Lucknow, Uttar Pr...",Candy Store,Burger Joint,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
14,"Dhaka, Model Town Tehsil",Furniture / Home Store,Pizza Place,Light Rail Station,Women's Store,Event Space,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
15,Model Town Tehsil,Hotel,Convenience Store,Coffee Shop,Bakery,Pizza Place,Women's Store,Event Space,Department Store,Dessert Shop,Dim Sum Restaurant
18,"Sector 7, Rohini, Alipur Tehsil",Convenience Store,Indian Restaurant,Café,Women's Store,Fast Food Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop
20,"Daryaganj, Old Delhi, Delhi, Kotwali Tehsil",History Museum,Road,Park,Cricket Ground,Fried Chicken Joint,Eastern European Restaurant,Dance Studio,Deli / Bodega,Department Store,Garden
22,Chanakya Puri Tehsil,Café,Coffee Shop,Indian Restaurant,Bar,Pub,Fast Food Restaurant,Asian Restaurant,Italian Restaurant,Lounge,Mediterranean Restaurant
23,Chanakya Puri Tehsil,Café,Coffee Shop,Indian Restaurant,Bar,Pub,Fast Food Restaurant,Asian Restaurant,Italian Restaurant,Lounge,Mediterranean Restaurant
24,"Rajendra Nagar, Karol Bagh Tehsil",Music Store,Market,Sporting Goods Shop,Food & Drink Shop,Café,Sandwich Place,Dance Studio,Deli / Bodega,Department Store,Dessert Shop


Fourth Cluster contains Park,Bakery,Ice Cream Shops,Pizza Place and Cafe. These places can be good for families with kids.

<b>Examine the fifth cluster</b>

In [175]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 4, delhi_merged.columns[[3] + list(range(5, delhi_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Jahangirpuri Colony, Model Town Tehsil",Electronics Store,Women's Store,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
9,"Banker, Narela Tehsil",Electronics Store,Women's Store,Fast Food Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
11,Civil Lines Tehsil,Platform,Electronics Store,Women's Store,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop


Fifth Cluster contains Electronics Store and Women's Store mostly. There are not so much neighbours in this cluster.

### In Mumbai

Analyse each of the clusters to identify the characteristics of each cluster and the neighborhoods in them.

<b>Examine the first cluster</b>

In [176]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0, mumbai_merged.columns[[3] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",Gym / Fitness Center,Hotel,Restaurant,Lounge,Indian Restaurant,Bank,Chinese Restaurant,Coffee Shop,Roof Deck,Dessert Shop
61,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",Movie Theater,Cricket Ground,Indian Restaurant,Grocery Store,Historic Site,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Fish Market
62,"E Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",Indian Restaurant,Café,Garden,Ice Cream Shop,Chinese Restaurant,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop
65,"K/E Ward, Zone 3, Mumbai, Mumbai Suburban, Mah...",Bakery,Bike Rental / Bike Share,Snack Place,Athletics & Sports,Yoga Studio,Fish Market,Field,Fast Food Restaurant,Farmers Market,Electronics Store
66,"K/W Ward, Zone 3, Mumbai, Mumbai Suburban, Mah...",Pub,Lounge,Multiplex,Indian Restaurant,Café,Asian Restaurant,Italian Restaurant,Bar,Brewery,Electronics Store
67,"Bandra West, Zone 3, Mumbai, Mumbai Suburban, ...",Indian Restaurant,Café,Coffee Shop,Chinese Restaurant,Gourmet Shop,Asian Restaurant,Snack Place,Bookstore,Clothing Store,Gym / Fitness Center
68,"K/E Ward, Zone 3, Mumbai, Mumbai Suburban, Mah...",Indian Restaurant,Ice Cream Shop,Fast Food Restaurant,Bistro,Pharmacy,Chinese Restaurant,South Indian Restaurant,Sports Bar,Pizza Place,Coffee Shop
69,"H/W Ward, Zone 3, Mumbai, Mumbai Suburban, Mah...",Clothing Store,Snack Place,Boutique,Bakery,Italian Restaurant,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Coffee Shop,Lounge
71,"K/W Ward, Zone 3, Mumbai, Mumbai Suburban, Mah...",Café,Beach,Ice Cream Shop,Pizza Place,Bar,Clothing Store,Italian Restaurant,Restaurant,Creperie,Sandwich Place
72,"H/W Ward, Zone 3, Mumbai, Mumbai Suburban, Mah...",Clothing Store,Chinese Restaurant,Accessories Store,Ice Cream Shop,Electronics Store,Men's Store,Café,Lounge,Food Truck,Furniture / Home Store


This cluster has a lot of neighbors. Some most common venues here are Restaurants,cafe,pub and bakery. This can be good for people who love to ate different foods as different restaurants are available here. Also snack place,Gym,Spa are somewhat famous here. 

<b>Examine the second cluster</b>

In [177]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1, mumbai_merged.columns[[3] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,"Panvel, Raigad, Maharashtra, 400056, India",Scenic Lookout,Diner,Fish Market,Field,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop,Yoga Studio,Convenience Store


This cluster contains only one neighbor. Some most common venues are Scenic lookout,Diner and Fish Market.

<b>Examine the third cluster</b>

In [178]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2, mumbai_merged.columns[[3] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,"F/N Ward, Zone 2, Mumbai, Mumbai City, Maharas...",Gym / Fitness Center,Playground,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop,Diner,Department Store
64,"B Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",Indian Restaurant,Furniture / Home Store,Chinese Restaurant,Indian Sweet Shop,Flea Market,Deli / Bodega,Department Store,Dessert Shop,Cupcake Shop,Fish Market
82,"C Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",Indian Restaurant,Jewelry Store,Train Station,Fast Food Restaurant,Bus Station,Café,Electronics Store,Fish Market,Field,Farmers Market
92,"G/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",Indian Restaurant,Seafood Restaurant,Bakery,Flower Shop,Fish Market,Field,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop


This cluster contains only three entries.Indian Restaurants are quite famous in this clusters. This place can be good for those who like to have Indian(or home-made) food only.

<b>Examine the fourth cluster</b>

In [179]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 3, mumbai_merged.columns[[3] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,"F/S Ward, Zone 2, Mumbai, Mumbai City, Maharas...",Gym,Rest Area,Convention Center,Diner,Field,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop,Dessert Shop


This cluster contains only one neighbor. Some most common venues are Gym,Rest Area and Convention Center.

<b>Examine the fifth cluster</b>

In [180]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 4, mumbai_merged.columns[[3] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,"E Ward, Zone 1, Mumbai, Mumbai City, Maharasht...",Furniture / Home Store,Pizza Place,Hookah Bar,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop,Diner,Dessert Shop


This cluster contains only one neighbor. Some most common venues are Furniture Store, Pizza Place, Hookah bar.

### In Chennai

Analyse each of the clusters to identify the characteristics of each cluster and the neighborhoods in them.

<b>Examine the first cluster</b>

In [181]:
chennai_merged.loc[chennai_merged['Cluster Labels'] == 0, chennai_merged.columns[[3] + list(range(5, chennai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,"Ward 63, Zone 5 Royapuram, சென்னை - Chennai, C...",Indian Restaurant,Electronics Store,Flea Market,Market,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop
94,"Ward 118, Zone 9 Teynampet, சென்னை - Chennai, ...",Indian Restaurant,Record Shop,Food Court,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market
96,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ...",Indian Restaurant,Hotel,Department Store,Juice Bar,Jewelry Store,Italian Restaurant,Ice Cream Shop,Fast Food Restaurant,Concert Hall,Park
98,"Ward 136, Zone 10 Kodambakkam, சென்னை - Chenna...",Indian Restaurant,Restaurant,Clothing Store,Coffee Shop,Boutique,Department Store,Dessert Shop,Market,Miscellaneous Shop,Music Venue
99,"Ward 120, Zone 9 Teynampet, சென்னை - Chennai, ...",Indian Restaurant,Juice Bar,Hyderabadi Restaurant,Arts & Crafts Store,Fruit & Vegetable Store,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market
100,"Ward 113, Zone 9 Teynampet, சென்னை - Chennai, ...",Indian Restaurant,Bookstore,Italian Restaurant,Movie Theater,Clothing Store,Café,Bus Station,South Indian Restaurant,Park,Hotel
104,"Ward 175, Zone 13 Adyar, சென்னை - Chennai, Che...",Indian Restaurant,Rock Club,Juice Bar,North Indian Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Movie Theater,Mediterranean Restaurant,Asian Restaurant,Fast Food Restaurant
106,"Ward 176, Zone 13 Adyar, சென்னை - Chennai, Che...",Indian Restaurant,Coffee Shop,Ice Cream Shop,Bistro,South Indian Restaurant,Fish Market,Pizza Place,Chinese Restaurant,Café,Donut Shop
112,"Ward 134, Zone 10 Kodambakkam, சென்னை - Chenna...",Indian Restaurant,Juice Bar,Lounge,Fried Chicken Joint,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market
115,"Ward 135, Zone 10 Kodambakkam, சென்னை - Chenna...",Indian Restaurant,Restaurant,Snack Place,Gym / Fitness Center,Grocery Store,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Hotel


In this cluster, Indian Restaurants are quite famous here.  This place can be good for those who like to have Indian(or home-made) food only.

<b>Examine the second cluster</b>

In [182]:
chennai_merged.loc[chennai_merged['Cluster Labels'] == 1, chennai_merged.columns[[3] + list(range(5, chennai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,"Ward 109, Zone 9 Teynampet, சென்னை - Chennai, ...",Burger Joint,Juice Bar,Bakery,Department Store,Hotel,Hotel Bar,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant
97,"Ward 111, Zone 9 Teynampet, சென்னை - Chennai, ...",Multiplex,Indian Restaurant,Café,Bakery,Juice Bar,Middle Eastern Restaurant,Movie Theater,Restaurant,Bus Station,Chinese Restaurant
101,"Ward 123, Zone 9 Teynampet, சென்னை - Chennai, ...",Clothing Store,Train Station,Indian Restaurant,Snack Place,Fried Chicken Joint,Flea Market,Buffet,Park,Hotel,Vegetarian / Vegan Restaurant
102,"Ward 123, Zone 9 Teynampet, சென்னை - Chennai, ...",Indian Restaurant,Furniture / Home Store,Café,Restaurant,Ice Cream Shop,Juice Bar,Kerala Restaurant,Fruit & Vegetable Store,Pharmacy,Gym / Fitness Center
103,"Ward 127, Zone 10 Kodambakkam, சென்னை - Chenna...",ATM,Bus Line,Metro Station,Hotel,Gym / Fitness Center,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market
105,"Ward 139, Zone 10 Kodambakkam, சென்னை - Chenna...",Indian Restaurant,Vegetarian / Vegan Restaurant,Multicuisine Indian Restaurant,Movie Theater,Middle Eastern Restaurant,Flower Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant
108,"Ward 171, Zone 13 Adyar, சென்னை - Chennai, Che...",College Cafeteria,Park,Snack Place,Flower Shop,Department Store,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant
111,"Ward 138, Zone 10 Kodambakkam, சென்னை - Chenna...",Indian Restaurant,Bakery,Chinese Restaurant,Mobile Phone Shop,Park,Grocery Store,Fish Market,Dessert Shop,Donut Shop,Electronics Store
113,"Ward 172, Zone 13 Adyar, சென்னை - Chennai, Che...",Park,Flower Shop,Bakery,Golf Course,Café,South Indian Restaurant,Department Store,Dessert Shop,Donut Shop,Electronics Store
117,"Ward 126, Zone 9 Teynampet, சென்னை - Chennai, ...",Pizza Place,Café,Snack Place,Park,Spa,Light Rail Station,Farmers Market,Department Store,Convenience Store,Gym / Fitness Center


This cluster contains a good number of neighbors.Some famous venues in this cluster are Indian Restaurants,Park,Hotel and Multiplex. This seems to be a market place.

<b>Examine the third cluster</b>

In [183]:
chennai_merged.loc[chennai_merged['Cluster Labels'] == 2, chennai_merged.columns[[3] + list(range(5, chennai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
110,"Ward 174, Zone 13 Adyar, சென்னை - Chennai, Che...",Indian Restaurant,Ice Cream Shop,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop


This cluster contains only one neighbor. Some most common venues are Indian Restaurant, Ice Cream Shop, Dessert Shop

<b>Examine the fourth cluster</b>

In [184]:
chennai_merged.loc[chennai_merged['Cluster Labels'] == 3, chennai_merged.columns[[3] + list(range(5, chennai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
109,"Ward 142, Zone 10 Kodambakkam, சென்னை - Chenna...",Hotel Bar,Train Station,Women's Store,Food Court,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market


This cluster contains only one neighbor. Some most common venues are Hotel Bar, Train Station and Women's Store.

<b>Examine the fifth cluster</b>

In [185]:
chennai_merged.loc[chennai_merged['Cluster Labels'] == 4, chennai_merged.columns[[3] + list(range(5, chennai_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
114,"Ward 128, Zone 10 Kodambakkam, சென்னை - Chenna...",Pizza Place,Clothing Store,Women's Store,Food Court,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market
124,"Ward 36, Zone 4 Tondiarpet, சென்னை - Chennai, ...",Pizza Place,Women's Store,Food Court,Dessert Shop,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market


There are only two neighbors in this clusters. Pizza Place, Women's Store and Food Court are quite famous here. This people might not be health concious.

### In Kolkata

Analyse each of the clusters to identify the characteristics of each cluster and the neighborhoods in them.

<b>Examine the first cluster</b>

In [186]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 0, kolkata_merged.columns[[3] + list(range(5, kolkata_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
128,"Sonarpur, South 24 Parganas, West Bengal, 7000...",Tea Room,Circus,Electronics Store,Bus Station,Tram Station,Food Court,Department Store,Dhaba,Dumpling Restaurant,Fast Food Restaurant
134,"Kolkata, West Bengal, 700032, India",Chinese Restaurant,Café,Cafeteria,Bistro,Tram Station,Food Court,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant
138,"Kolkata, West Bengal, 700047, India",Pizza Place,Playground,Supermarket,Food,Cosmetics Shop,Cricket Ground,Department Store,Dhaba,Dumpling Restaurant,Electronics Store
146,"Kolkata, West Bengal, 700053, India",Indian Sweet Shop,Asian Restaurant,Chinese Restaurant,Snack Place,Historic Site,History Museum,Department Store,Dhaba,Dumpling Restaurant,Hotel
151,"Behala, Kolkata, West Bengal, 700060, India",Health & Beauty Service,Auto Dealership,Chinese Restaurant,Tram Station,Food Court,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant
158,"Kolkata, West Bengal, 700014, India",Chinese Restaurant,Bus Station,Tram Station,Food Court,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
159,"Bow Bazaar, Kolkata, West Bengal, 700012, India",Hotel,Chinese Restaurant,Café,Tram Station,Food Court,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant
169,"Kolkata, West Bengal, 700054, India",Indian Restaurant,Indian Sweet Shop,Café,Dhaba,Chinese Restaurant,Pizza Place,Harbor / Marina,Gym,Hotel,Department Store
177,"Kolkata, West Bengal, 700006, India",History Museum,Asian Restaurant,Chinese Restaurant,Café,Sporting Goods Shop,Tram Station,Food Court,Dhaba,Dumpling Restaurant,Electronics Store
179,"Kolkata, West Bengal, 700002, India",IT Services,Electronics Store,Burger Joint,Cosmetics Shop,Department Store,Dhaba,Dumpling Restaurant,Fast Food Restaurant,Field,Flea Market


In this cluster Chinese Restaurant, Pizza Place,Indian Sweet Shop and Cafe are quite famous. 

<b>Examine the second cluster</b>

In [187]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 1, kolkata_merged.columns[[3] + list(range(5, kolkata_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
130,"Kolkata, West Bengal, 700008, India",ATM,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field,Flea Market
139,"Kolkata, West Bengal, 700024, India",ATM,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field,Flea Market
140,"Sonarpur, South 24 Parganas, West Bengal, 7000...",ATM,Chinese Restaurant,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
142,"Kolkata, West Bengal, 700078, India",ATM,Photography Studio,Department Store,Food Stand,Cricket Ground,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
147,"Thakurpukur Mahestola, South 24 Parganas, West...",ATM,Bus Station,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
154,"Sonarpur, South 24 Parganas, West Bengal, 7000...",ATM,Hotel,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
157,"Kolkata, West Bengal, 700043, India",ATM,Harbor / Marina,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
171,"Kolkata, West Bengal, 700010, India",ATM,Chinese Restaurant,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
174,"Barrackpur - II, North 24 Parganas, West Benga...",ATM,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field,Flea Market
178,"Kolkata, West Bengal, 700037, India",ATM,Dhaba,Optical Shop,Tram Station,Food Stand,Department Store,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field


In this cluster ATM,Train Stand and Food Stand are quite famous here. This place can be good for those working professionals who are relocating for their job and have to travel often

<b>Examine the third cluster</b>

In [188]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 2, kolkata_merged.columns[[3] + list(range(5, kolkata_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
141,"Kolkata, West Bengal, 700095, India",Spa,Tram Station,Food Stand,Cricket Ground,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field


This cluster contains only one neighbor. Some most common venues are Spa,Train Station and Food Stand.

<b>Examine the fourth cluster</b>

In [189]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 3, kolkata_merged.columns[[3] + list(range(5, kolkata_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
144,"Behala, Kolkata, West Bengal, 700061, India",Jewelry Store,Market,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
149,"Kolkata, West Bengal, 700023, India",ATM,Awadhi Restaurant,Market,Tram Station,Furniture / Home Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field
183,"Lake Town, Khardaha, Barrackpur - II, North 24...",Mobile Phone Shop,Market,IT Services,Historic Site,Health & Beauty Service,Cricket Ground,Department Store,Dhaba,Dumpling Restaurant,Electronics Store
187,"Barrackpur - II, North 24 Parganas, West Benga...",Mobile Phone Shop,American Restaurant,Music Store,Tram Station,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field


This cluster contains only four neighbors. Some of the famous venues here are Mobile Shop, Market and Train Station.

<b>Examine the fifth cluster</b>

In [169]:
kolkata_merged.loc[kolkata_merged['Cluster Labels'] == 4, kolkata_merged.columns[[1] + list(range(5, kolkata_merged.shape[1]))]]

,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
125,88.329496,Vegetarian / Vegan Restaurant,Garden,South Indian Restaurant,Food Court,Cricket Ground,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant
126,88.358539,Indian Restaurant,History Museum,Art Museum,Museum,Food Court,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant
127,88.348456,Nightclub,Bengali Restaurant,American Restaurant,Café,Fast Food Restaurant,Department Store,Dhaba,Multiplex,Plaza,Restaurant
129,88.329553,ATM,Historic Site,Food,Tram Station,Food Stand,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant
132,88.318691,Mughlai Restaurant,Tram Station,Indian Restaurant,History Museum,Historic Site,Cricket Ground,Department Store,Dhaba,Dumpling Restaurant,Electronics Store
133,88.347195,Indian Sweet Shop,Grocery Store,Clothing Store,Café,Mobile Phone Shop,Health & Beauty Service,Harbor / Marina,Dhaba,Hotel,Dumpling Restaurant
135,88.366080,Bakery,Bengali Restaurant,Indian Sweet Shop,Sports Club,Dhaba,Flea Market,Pizza Place,Plaza,Vegetarian / Vegan Restaurant,Sporting Goods Shop
136,88.368115,Plaza,Café,Bengali Restaurant,Train Station,Chinese Restaurant,Market,Mughlai Restaurant,Boutique,Tram Station,Food
137,88.409922,Coffee Shop,Food Stand,Soccer Field,History Museum,Historic Site,Cricket Ground,Department Store,Dhaba,Dumpling Restaurant,Electronics Store
143,88.349475,Café,Stadium,Movie Theater,Tram Station,Food Court,Department Store,Dhaba,Dumpling Restaurant,Electronics Store,Fast Food Restaurant


This cluster contains a good number of neighbors.Some of the famous venues here are Hotel,Dhaba,Park,Cafe,Restaurants and Market. This places are also good for foodies or travellers.

## Results and Discussion

In this project, We attempted to load dataset of India's metropolitan cities and try to analyze neighbourhood in these metro cities based on most popular venues they have. We used k.means clustering algortihm to cluster the neighbourhood. 

The main aim of this project was to help people to relocate or settle in these metro cities.Given the cluster informations of all metro cities.

We can say that <b>Delhi</b> people are good for art,craft,playgrounds and malls.Also delhi contains Indian Restaurants and Fast food places.So, it might not be good international visitors.

In <b>Mumbai</b> Cafe,Pub,Gym and Spa are quite famous here. These people are more health concious. Also due to close proximity to sea shore, fish markets are also famous there.

<b>Chennai</b> and its neighborhoods are a great place for foodie. There are various types of restaurants and hotels in chennai.
    
<b>Kolkata</b> and its neighborhoods contain Shopping malls,Multiplex,Atms etc.It contains Park,Gyms,Cafe and Hotel too.

We can also analyse each neighborhood cluster wise to find the best neighborhood for targeted persons

## Conclusion

This project helps us to get better understandings of neighborhoods with respect to most common venues.

The future of this project can be to include data related to crimes,pricing and more to get better insights of the neighborhoods and suggest better neighborhood.